# Homework2 - Is it edible?

reference: [Mushrroms](https://www.kaggle.com/uciml/mushroom-classification)

這個是一個關於蘑菇的Dataset,裏面記錄蘑菇的特徵(Feature),像是能不能吃及蘑菇顏色,蕈柄等

# Problems

藉由蘑菇的各種特徵,判別這個蘑菇可不可以吃

以下開工

# Features

我們使用幾個features

### class: [e,p]

e=edible,p=poisonous

### cap_color:[n,b,c,g,r,p,u,e,w,y]

brown=n,buff=b,cinnamon=c,gray=g,green=r,pink=p,purple=u,red=e,white=w,yellow=y

### cap_odor:[a,l,c,y,f,m,n,p,s]

almond=a,anise=l,creosote=c,fishy=y,foul=f,musty=m,none=n,pungent=p,spicy=s

### cap_shape:[b,c,x,f,k,s]

bell=b,conical=c,convex=x,flat=f, knobbed=k,sunken=s

### cap_surface:[f,g,y,s]

fibrous=f,grooves=g,scaly=y,smooth=s


In [167]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals.six import StringIO   
from sklearn.tree import export_graphviz
import pydotplus


cap_color=['p','u','e','n','w','c','b','g','y']
cap_odor=['a','l','n','y','f','m','c','p','s']
cap_shape=['b','c','x','f','k','s']
cap_surface=['f','g','y','s']
cap_class=['e','p']

# Rules

以下是幾個rules,後者會把前者值取代

1. if cap_odor='a','l','n' class=e else class=p
2. if cap_odor='n' and cap_color='p','u','e' class=p else class=e
3. if cap_odor='n' and cap_color='n','w','c' and cap_shape='s','x' class=p else class=e
4. if cap_odor='n' and cap_color='b','g','y','r' and cap_surface='f','s' class=e else class=p

接下來就依rules 產生資料

In [168]:
col_color=[]
col_odor=[]
col_shape=[]
col_surface=[]
col_class=[]

# create data
# all class are set to edible (will be changed by rules later)
for color in cap_color:
    for odor in cap_odor:
        for shape in cap_shape:
            for surface in cap_surface:
                col_color.append(color)
                col_odor.append(odor)
                col_shape.append(shape)
                col_surface.append(surface)
                col_class.append('e')

d = {'cap_color':col_color, 'cap_odor':col_odor,'cap_shape':col_shape,'cap_surface':col_surface,'cap_class':col_class}
df = pd.DataFrame(data=d)
df.head()

,cap_color,cap_odor,cap_shape,cap_surface,cap_class
0,p,a,b,f,e
1,p,a,b,g,e
2,p,a,b,y,e
3,p,a,b,s,e
4,p,a,c,f,e


In [169]:
# Define edible and poisonous by rules
for index, row in df.iterrows():
    
    if(row['cap_odor']=='a'or row['cap_odor']== 'l' or row['cap_odor']== 'n'):
        row['cap_class']='e'
    else:
        row['cap_class']='p'
    
    if(row['cap_odor']=='n'):
        if(row['cap_color']=='p' or row['cap_color']=='u' or row['cap_color']=='e'):
            row['cap_class']='p'
        else:
            row['cap_class']='e'
        
        if(row['cap_color']=='n' or row['cap_color']=='w' or row['cap_color']=='c'):
            if(row['cap_shape']=='s'or row['cap_shape']=='x' ):
                row['cap_class']='p'
            else:
                row['cap_class']='e'
        
        if(row['cap_color']=='b' or row['cap_color']=='g' or row['cap_color']=='y' or row['cap_color']=='r'):
            if(row['cap_surface']=='f' or row['cap_surface']=='s'):
                row['cap_class']='e'
            else:
                row['cap_class']='p'

In [170]:
#convert string to value
for index, row in df.iterrows():
    row['cap_color']= np.float32(cap_color.index(row['cap_color']))
    row['cap_odor']= np.float32(cap_odor.index(row['cap_odor']))
    row['cap_shape']= np.float32(cap_shape.index(row['cap_shape']))
    row['cap_surface']= np.float32(cap_surface.index(row['cap_surface']))
    row['cap_class']=np.float32(cap_class.index(row['cap_class']))

# drop the answers
y = df['cap_class'].values
df= df.drop('cap_class', 1)

# fit tree
y=y.astype('int')
dtree=DecisionTreeClassifier()
dtree.fit(df,y)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [171]:
#create PDF
dot_data = StringIO()
export_graphviz(dtree, 
                out_file=dot_data,  
                filled=True, 
                feature_names=list(df),
                class_names=['cap_class','cap_shape','cap_odor','cap_surface','cap_color'],
                special_characters=True)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_pdf("tree.pdf")

True

會產生一份為`tree.pdf`的decision tree圖片

這樣子作業基本上就結束了

# 討論

### 1.這張圖其實是已經經過美化的結果

在製作的時候發現classifier是用value去做if的decision tree

所以如果在最一開始時把array的element先做好排序的話tree會很精緻

反之如果沒有的話tree會變得非常大

### 2.Tree 的 gini到最底時都是0.0

Decision tree的最底都是0.0,可以說是都fit data,這是因為我們的decision tree沒有限制depth

在沒有限制的depth下有可能會導致training data overfitting的現象

### 3.Classifier決策與我們預定判斷不同

我們的rule主要是 feature='a','b','c'這類型的方式,轉為number會變成1<=feature<=3這類的判斷

但是classifier的判斷是feature<=3再去做判斷,而不是一個區間

所以接下來的判斷不一定是feature>=1而可能是看哪個feature好區分出兩個東西